In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0 

--2025-05-25 10:12:28--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0 [following]
--2025-05-25 10:12:29--  https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7c54d8f8cecd9cc371de9868c4.dl.dropboxusercontent.com/zip_download_get/CNv1nW7U2K9BS1cX-OCUyDjX1XDU-iRPaSxuhjnXVsRAHcbtEhfag4NdyMFRD1Bwkbs-pPTkewLUH2wCeHR-8Zhj4qKjEhF4TuWPC52JVOLblg# [following]
--2025-05-25 10:12:29--  https://uc7c54d8f8cecd9cc371de9868c4.dl.dropboxusercontent.com/zip_

In [3]:
!unzip AAAw1_JIzpuVvwteJCma0xMla?dl=0

Archive:  AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [4]:
df=pd.read_csv('train.csv', encoding='ISO-8859-1')
df.head(10)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
5,00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,00005059a06ee19e11ad,Why does Quora automatically ban conservative ...,0
7,0000559f875832745e2e,Is it crazy if I wash or wipe my groceries off...,0
8,00005bd3426b2d0c8305,"Is there such a thing as dressing moderately, ...",0
9,00006e6928c5df60eacb,Is it just me or have you ever been in this ph...,0


In [5]:
def preprocess_text(raw_text):
    cleaned = re.sub(r'[^a-zA-Z0-9 ]', '', raw_text)
    return cleaned.lower()

df['question_text'] = df['question_text'].apply(preprocess_text)

In [7]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
texts = df['question_text']
labels = df['target']
texts_train, texts_val, labels_train, labels_val = train_test_split(texts, labels, test_size=0.2, random_state=42)

2025-05-25 10:16:02.303634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748168162.557975      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748168162.626559      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [9]:
# Tokenize
max_vocab = 20000
seq_len = 100

text_tokenizer = Tokenizer(num_words=max_vocab)
text_tokenizer.fit_on_texts(texts_train)

seq_train = text_tokenizer.texts_to_sequences(texts_train)
seq_val = text_tokenizer.texts_to_sequences(texts_val)

padded_train = pad_sequences(seq_train, maxlen=seq_len)
padded_val = pad_sequences(seq_val, maxlen=seq_len)

In [10]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2025-05-25 10:18:14--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2025-05-25 10:18:15--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2025-05-25 10:18:15--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [11]:
!unzip glove.42B.300d.zip
!rm glove.42B.300d.zip

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [12]:
# Load GloVe embeddings
embedding_index = {}
embedding_dim = 120
glove_file = 'glove.42B.300d.txt'
with open(glove_file, 'r', encoding='utf-8') as emb_file:
    for record in emb_file:
        values = record.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

In [13]:
# Prepare embedding matrix
word_map = text_tokenizer.word_index
embedding_dim = 300
embedding_matrix = np.zeros((max_vocab, embedding_dim))
for token, idx in word_map.items():
    if idx < max_vocab:
        vec = embedding_index.get(token)
        if vec is not None:
            embedding_matrix[idx] = vec


In [14]:
# Build model
model = Sequential([Embedding(input_dim=max_vocab, output_dim=embedding_dim,
              weights=[embedding_matrix], input_length=seq_len, trainable=False),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-05-25 10:38:23.786959: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       6,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,000,000 (22.89 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 6,000,000 (22.89 MB)

None


In [17]:
# Train
batch_size = 32
epochs = 5
result = model.fit(padded_train, labels_train,
                    validation_data=(padded_val, labels_val),
                    batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 5105s 156ms/step - accuracy: 0.9501 - loss: 0.1309 - val_accuracy: 0.9580 - val_loss: 0.1067
Epoch 2/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 5111s 157ms/step - accuracy: 0.9568 - loss: 0.1102 - val_accuracy: 0.9589 - val_loss: 0.1049
Epoch 3/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 5144s 158ms/step - accuracy: 0.9584 - loss: 0.1055 - val_accuracy: 0.9594 - val_loss: 0.1027
Epoch 4/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 5138s 157ms/step - accuracy: 0.9592 - loss: 0.1023 - val_accuracy: 0.9600 - val_loss: 0.1046
Epoch 5/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 5118s 157ms/step - accuracy: 0.9602 - loss: 0.1001 - val_accuracy: 0.9592 - val_loss: 0.1033


In [18]:
# Evaluate
from sklearn.metrics import classification_report
pred = (model.predict(padded_val) > 0.5).astype(int)
print(classification_report(labels_val, pred))

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 272s 33ms/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98    245369
           1       0.68      0.62      0.65     15856

    accuracy                           0.96    261225
   macro avg       0.83      0.80      0.81    261225
weighted avg       0.96      0.96      0.96    261225

